In [65]:
# Load relevant packages
from bs4 import BeautifulSoup # For parsing html
import requests # makes http requests

from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool 
import time
import threading
from queue import Queue

import csv

In [125]:
'''Specify the URLS'''
URL_MAIN = "http://www.eixgrandegracia.cat"
URL_STORES = "http://www.eixgrandegracia.cat/web/comercios"

In [132]:
# Functions used for parsing 
def mapping_parser(list_of_urls): 
    results = list(map(parser, list_of_urls))
    return results

def parser(url):
    my_info = get_info(url)[0] # gets the info for this store
    return my_info

def website_directed(url_end): # Concatenates the domain of the website with the directed URL
    website = "{}{}".format(URL_MAIN, url_end)
    return website 

def url_list(url):
    urls = []
    for i in range(len(get_soup(url).select("div.force-box a"))): # searches the whole range of images that direct you to the stores info url
        urls.append(get_soup(url).select("div.force-box a")[i]['href']) # gets the url of the store
    return(urls)

def get_soup(url):
    res = requests.get(url) # gets a webpage <Response [200]>
    return BeautifulSoup(res.text, "lxml") # transforms the html code of the requested url in nice for python format

def get_info(url):
    info_one = [] # get's info from a signle item, e.g. the name of the store
    info_all = [] # combines all info of one store
    for j in range(len(get_soup(url).select("label.col-md-9"))): 
        title = " ".join(str.split(get_soup(url).select("div.col-sm-11 h3")[0].text)) # gets the name of the store in nice format
        info_one.append(" ".join(str.split(get_soup(url).select("label.col-md-9")[j].text))) # gets the rest of the info in nice format
    info_all.append({'Name': title, 'URL': info_one[0], 'Product': info_one[1], 'Telephone': info_one[2], "Adress": info_one[3]}) # creates a dictionary with all the info of the store
    return(info_all) 
                
def make_csv(list_of_dictionaries):
    keys = list_of_dictionaries[0].keys() # gets the keys of the list of dictionaries
    with open('stores_info.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys) # creates an object which operates like a regular writer but maps dictionaries onto output rows.
        dict_writer.writeheader() # writes a row with the field names
        dict_writer.writerows(list_of_dictionaries) # writes all elements in rows

In [ ]:
'''Trial'''
result = list(map(website_directed, url_list(URL_STORES)))
mapping_parser(result)

In [5]:
'''For my knowledge'''
list_of_apples = [1,2,3,4,5]

def toyfun(a):
    return a * 2

#for i in range(len(list_of_apples)):
#    list_of_apples[i] = toyfun(list_of_apples[i]) 

with ThreadPoolExecutor(50) as pool:
    results = pool.map(toyfun, list_of_apples)
print(list(results))  

[2, 4, 6, 8, 10]
